# Cell type assignment

In this notebook, we analyse the cell type predictions of the automated annotation model and validate its cell type assignments using marker gene expression and other sources of information (e.g. knowledge of developmental stages and anatomical dissections).  

In [147]:
import scrabbitpy
import scanpy as sc
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


### Load data

In [66]:
# Load data
r_data = sc.read_h5ad("../data-in/cell_type_annotation/rabbit_corrected_clustered.h5ad")
orthologs = pd.read_csv("../data-in/orthologs/rabbit_mouse_orthologs.tsv", sep="\t")


In [50]:
m_markers = pd.read_csv("../data-in/cell_type_annotation/celltype_markers.tsv", sep="\t")
m_markers = {i: m_markers[i].dropna().to_list() for i in m_markers.columns}


### Cluster data

Cell type labels are assigned to clusters identified in the high-dimensional gene expression space. Here we perfrom Leiden clustering at various resolutions to identify both coarse and fine-grained populations of cells.

For the sake of brevity, here we have used precomputed clusterings. See `scripts/cluster.py` to run the full clustering.  

In [ ]:
# Load clusters from file
clusters = pd.read_csv("../data-in/cell_type_annotation/clusters.tsv",sep="\t")
r_data.obs = r_data.obs.join(clusters)


### Divide dataset

To simplify the annotation process, the dataset is initially divded into broad regions to be annotated independently. 

For each broad region, we plot the automated annotation predictions, along with clusterings of different resolutions. In addition to clustering the entire dataset, Leiden clustering is also performed within each region. These results are compared to UMAP plots of marker gene expression of known cell types as well as differentially expressed genes computationally identified in the annotated mouse dataset.



In [ ]:
blood = rabbit[rabbit.obs["leiden_res1"].isin(['4','19','20','8']),]
mesoderm = rabbit[rabbit.obs["leiden_res1"].isin(['17','2','12','22']),]
neural = rabbit[rabbit.obs["leiden_res1"].isin(['3','7']),]
misc = rabbit[rabbit.obs["leiden_res1"].isin(['6','23','21','18']),]
exe_ectoderm = rabbit[rabbit.obs["leiden_res1"].isin(['9','0','16']),]
exe_mesoderm = rabbit[rabbit.obs["leiden_res1"].isin(['5','14','10','1','22']),]
exe_endoderm = rabbit[rabbit.obs["leiden_res1"].isin(['13','15','11']),]



In [79]:
# create directory for each region
# Identify mouse cell types likely to be in each 


In [74]:
def createAnnotationDirs(base_path, clusters):
    """
    
    e.g.
    createAnnotationDirs(base_path = "../figs/celltype_annotation/annotation_pipeline/r_mesoderm/", 
                     clusters = ["leiden_res8","mesoderm_leiden_res2","mesoderm_leiden_res5"])
                     
    """
    
    # Marker expression plots
    Path(base_path + "marker_expression").mkdir(parents=True, exist_ok=True)
    
    # Cluster fraction plots
    Path(base_path + "cluster_fractions").mkdir(parents=True, exist_ok=True)
    
    # Prediction fraction plots
    Path(base_path + "cluster_fractions").mkdir(parents=True, exist_ok=True)
    
    for cluster in clusters:
        # Create directory for each clustering
        Path(base_path + "cluster_fractions/" + cluster).mkdir(parents=True, exist_ok=True)
        Path(base_path + "prediction_fractions/" + cluster).mkdir(parents=True, exist_ok=True)
        
    

In [ ]:
def plotMarkerExpression(adata, markers,):
    """
    adata: Scanpy object
    markers: Dictionary where keys are the cell types/obs groups of interest and items are marker gene names which match values in adata.var.index.   
    """
    
    

In [219]:
def computeObsFraction(adata, obsA="leiden_res8",obsB="singler", obsB_ignore_thresh = None ):
    """
    Computes the fraction of observations B within groups of observation A.
    
    """
    
    df = pd.DataFrame({obsA:adata.obs[obsA], obsB:adata.obs[obsB]},index=adata.obs.index)
    df[obsA + "_ncells"] = df.groupby([obsA]).transform('count')

    df[obsB + "_count"] = 0

    counts = df.groupby([obsA,obsB,obsA + "_ncells"]).count()
    counts[obsB + "_frac"] = counts[obsB + "_count"].groupby(obsA).transform(lambda x: x/x.sum())
    counts.sort_values([obsA,obsB,obsB + "_frac"],ascending=False)
    
    counts= counts.reset_index()
    counts = counts.dropna()
    
     # Group observations B with very few cells as 'Other'
    if(obsB_ignore_thresh is not None) :
        counts[obsB] = counts[obsB].astype(str)
        counts.loc[counts[obsB + "_frac"] <= obsB_ignore_thresh,obsB] = "Other"      
        
    
    
    return(counts)

In [227]:
data = computeObsFraction(r_mesoderm, obsA = "leiden_res8", obsB = "singler", obsB_ignore_thresh = 0.05)
data

,leiden_res8,singler,leiden_res8_ncells,singler_count,singler_frac
1066,3,Non-neural ectoderm 4,1,1.0,1.000000
1931,6,Caudal mesoderm,37,19.0,0.513514
2243,6,Other,37,1.0,0.027027
2425,6,Lateral plate mesoderm,37,8.0,0.216216
3023,6,Presomitic mesoderm,37,9.0,0.243243
...,...,...,...,...,...
47306,79,Sclerotome,294,25.0,0.085034
47332,79,Other,294,5.0,0.017007
47358,79,Other,294,9.0,0.030612
47410,79,Other,294,1.0,0.003401


In [228]:
data.loc[data[obsA + "_ncells"] >= 50]

,leiden_res8,singler,leiden_res8_ncells,singler_count,singler_frac
6587,11,Other,64,3.0,0.046875
6639,11,Other,64,1.0,0.015625
6743,11,Other,64,1.0,0.015625
6769,11,Cardiopharyngeal progenitors FHF,64,9.0,0.140625
6795,11,Cardiopharyngeal progenitors SHF,64,4.0,0.062500
...,...,...,...,...,...
47306,79,Sclerotome,294,25.0,0.085034
47332,79,Other,294,5.0,0.017007
47358,79,Other,294,9.0,0.030612
47410,79,Other,294,1.0,0.003401


In [201]:
ncells = df.groupby([obsA,obsA + "_ncells"]).count()
np.where(ncells[obsB] <= 0.05)

(array([], dtype=int64),)

In [212]:
#test[obsB].cat.add_categories("Other", inplace=True)
#test.loc[test[obsB + "_frac"] <= 0.05,obsB] = "Other"
test

,leiden_res8,singler,leiden_res8_ncells,singler_count,singler_frac
1066,3,Non-neural ectoderm 4,1,1.0,1.000000
1905,6,Caudal mesoderm,37,19.0,0.513514
2217,6,Other,37,1.0,0.027027
2399,6,Lateral plate mesoderm,37,8.0,0.216216
2997,6,Presomitic mesoderm,37,9.0,0.243243
...,...,...,...,...,...
46578,79,Sclerotome,294,25.0,0.085034
46604,79,Other,294,5.0,0.017007
46630,79,Other,294,9.0,0.030612
46682,79,Other,294,1.0,0.003401


In [ ]:
def plotObsFraction()

In [229]:
def plotMultiObsFraction(data, obsA, obsB, obsA_ignore_thresh = None, obsB_ignore_thresh = None, export_dir=None, obsB_colours=None):
    
    
    # Get obs fractions if anndata object passed in
    if(isinstance(r_mesoderm,sc.AnnData)):
        data = computeObsFraction(data, obsA = obsA, obsB = obsB, obsB_ignore_thresh = obsB_ignore_thresh)
    
    # Don't plot fractions for observations with very few cells
    if(obsA_ignore_thresh is not None):
        data = data.loc[data[obsA + "_ncells"] >= obsA_ignore_thresh]
    
    
    # if(export_dir is None):
        # Plot in a grid
    
    
    if(export_dir is not None):
        for x in data[obsA].unique():
            fig, ax = plt.subplots(figsize=(10,10))
            
            df_plot = data[data[obsA]==x]
            
            if(obsB_colours is not None):
                obsB_colours = [obsB_colours[i] for i in df_plot[obsB]]
                
            ax.barh(df_plot[obsB], df_plot[obsB + "_frac"], color=obsB_colours)
            
            ax.set_title(obsA + ": " +  x + " (" + str(int(df_plot[obsA + "_ncells"].iloc[0]))+ " cells)")
            ax.set_xlabel("Fraction of cells")
            ax.set_ylabel(obsB)
            
            plt.tight_layout()
            plt.savefig(export_dir + x + ".pdf")
            plt.close(fig)

    
    
    

In [230]:
plotMultiObsFraction(r_mesoderm, obsA = "leiden_res8", obsB = "singler", 
                     obsA_ignore_thresh = 50 , obsB_ignore_thresh = 0.02,
                     
                     #colours = scrabbit.getCelltypeColours(),
                     export_dir = "../figs/celltype_annotation/annotation_pipeline/r_mesoderm/cluster_fractions/leiden_res8/")

In [ ]:
def makeAnnotationPlots(adata, clusters, model_predictions, celltypes, markers, export_dir):
    
    
    createAnnotationDirs(base_path = export_dir, clusters = clusters )
    
    
    # Plot UMAPs of SingleR predictions, Clusterings
    sc.settings.figdir = export_dir
    sc.pl.umap(adata, color=[model_predictions,clusters],
               save="_clusters.pdf")
    

    # Plot UMAPs of literature marker genes
    for celltype in celltypes:
        sc.settings.figdir = export_dir + "marker_expression/"
        sc.pl.umap(adata,color=markers[celltype],
                   save="_" + celltype + "_markers.pdf")
    
   
    # Plot bar charts of cluster prediction fractions
    for cluster in clusters:
        sc.settings.figdir = export_dir + "cluster_fractions/" + cluster
        
        plotClusterFraction(adata, clusters=cluster, predictions=model_predictions, 
                            save= export_dir + "cluster_fractions/" + cluster)
        
    
    # Plot bar charts of prediction cluster fractions
    sc.settings.figdir = export_dir + "cluster_fractions/"
        
    plotClusterFraction(r_meosderm, clusters="leiden_res8", predictions="singler", save=True)


### Mesoderm

In [ ]:
# Create directory for mesoderm region
Path("../figs/celltype_annotation/annotation_pipeline/r_mesoderm/").mkdir(parents=True, exist_ok=True)

runAnnotationPlots(r_mesoderm, 
                    clusters = ["leiden_res8","mesoderm_leiden_res2","mesoderm_leiden_res5"],
                    model_predictions = "singler",
                   marker_genes = m_markers,
                   export_dir = "../figs/celltype_annotation/annotation_pipeline/r_mesoderm/")